In [1]:
pip install lazypredict

  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached markdown-3.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached pyarrow-19.0.1-cp39-cp39-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached sqlalchemy-2.0.41-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached databricks_sdk-0.55.0-py3-none-any.whl.metadata (39 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.33.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached sqlparse-0.5.3-py3-none-

In [1]:
from lazypredict.Supervised import LazyClassifier

In [2]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd


In [ ]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Path to preprocessed .pkl EEG files
data_dir = "preprocessed_epochs"
all_files = sorted(glob(os.path.join(data_dir, "*_epochs.pkl")))

# Group .pkl files by subject (assuming filenames like session1_sub01_epochs.pkl)
subject_sessions = {}
for file in all_files:
    basename = os.path.basename(file)
    subject_id = basename.split("_")[1]  # e.g., sub01
    subject_sessions.setdefault(subject_id, []).append(file)

# Dictionary to store results
all_results = {}

for subject, files in subject_sessions.items():
    X_list, y_list = [], []
    print(f"\n🔄 Processing subject: {subject}")

    for file in files:
        try:
            X, y = joblib.load(file)
            X_list.append(X)
            y_list.append(y)
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")
            continue

    if len(X_list) == 0:
        print(f"⚠️ No valid data found for {subject}")
        continue

    # Combine 3 sessions
    X_subject = np.concatenate(X_list)
    y_subject = np.concatenate(y_list)

    # Flatten each EEG sample from (channels, timepoints) to 1D
    X_flat = X_subject.reshape(X_subject.shape[0], -1)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_flat)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_subject, test_size=0.2, random_state=42)

    # LazyPredict classification
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, _ = clf.fit(X_train, X_test, y_train, y_test)

    print(f"📊 Classification results for {subject}:")
    print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    all_results[subject] = models

# Save results to CSV (optional)
summary_df = pd.concat([df.assign(Subject=subj) for subj, df in all_results.items()])
summary_df.to_csv("lazy_subjectwise_results.csv")
print("\n✅ All subject results saved to lazy_subjectwise_results.csv")


🔄 Processing subject: sub10


  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Path to preprocessed .pkl EEG files
data_dir = "preprocessed_epochs"
all_files = sorted(glob(os.path.join(data_dir, "*_epochs.pkl")))

# Group .pkl files by subject (assuming filenames like session1_sub01_epochs.pkl)
subject_sessions = {}
for file in all_files:
    basename = os.path.basename(file)
    subject_id = basename.split("_")[1]  # e.g., sub01
    subject_sessions.setdefault(subject_id, []).append(file)

# Dictionary to store results
all_results = {}

for subject, files in subject_sessions.items():
    print(f"\n🔄 Processing subject: {subject}")
    X_all, y_all = [], []

    for file in files:
        try:
            X, y = joblib.load(file)
            # Flatten and scale this session's data
            X_flat = X.reshape(X.shape[0], -1)
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_flat)

            # Store only temporarily
            X_all.append(X_scaled)
            y_all.append(y)
            
            # Immediately delete large variables to reduce RAM
            del X, y, X_flat
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")

    # Process subject if valid data found
    if len(X_all) == 0:
        print(f"⚠️ No valid data for {subject}")
        continue

    for X_sub, y_sub in zip(X_all, y_all):
        try:
            X_train, X_test, y_train, y_test = train_test_split(X_sub, y_sub, test_size=0.2, random_state=42)
            clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
            models, _ = clf.fit(X_train, X_test, y_train, y_test)

            if subject not in all_results:
                all_results[subject] = []
            all_results[subject].append(models)

            # Clear to save memory
            del X_train, X_test, y_train, y_test, X_sub, y_sub
        except Exception as e:
            print(f"❌ LazyClassifier failed for {subject}: {e}")

# Save results to CSV (optional)
summary_df = pd.concat([df.assign(Subject=subj) for subj, df_list in all_results.items() for df in df_list])
summary_df.to_csv("lazy_subjectwise_results.csv")
print("\n✅ All subject results saved to lazy_subjectwise_results.csv")



🔄 Processing subject: sub10


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 41, number of negative: 39
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.678154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037200
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.512500 -> initscore=0.050010
[LightGBM] [Info] Start training from score 0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.690610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036789
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

...

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 43, number of negative: 37
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.705439 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036857
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.537500 -> initscore=0.150282
[LightGBM] [Info] Start training from score 0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.659893 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036960
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

  0%|          | 0/32 [00:00<?, ?it/s]

In [1]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyClassifier
import pandas as pd

In [2]:
#ONLY ONE SUBJECT

import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
#from lazypredict.Supervised import LazyClassifier

# ----------- CONFIGURE THIS ----------
subject_id = "1"  # e.g., 'sub1', 'sub10', etc.
data_dir = "preprocessed_epochs"
# -------------------------------------

# Get all 3 session files for this subject
subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

X_list, y_list = [], []

for file in subject_files:
    try:
        print(f"📂 Loading: {file}")
        X, y = joblib.load(file)
        X_list.append(X)
        y_list.append(y)
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Check if data was loaded
if not X_list:
    print("❌ No data loaded. Check subject ID or file path.")
else:
    # Process subject data
    for i in range(len(X_list)):
        # Reshape & standardize session data
        X_flat = X_list[i].reshape(X_list[i].shape[0], -1)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_flat)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_list[i], test_size=0.2, random_state=42)

        print(f"\n🚀 Running LazyPredict on Session {i+1} of {subject_id}...")
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        print(f"📊 Results for session {i+1} of {subject_id}:\n")
        print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

📂 Loading: preprocessed_epochs/session1_sub1_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session2_sub1_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session3_sub1_twist_MI_epochs.pkl

🚀 Running LazyPredict on Session 1 of 1...


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.696818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037108
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.684637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036750
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.705722 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037201
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [3]:
# ----------- CONFIGURE THIS ----------
subject_id = "6"  # e.g., 'sub1', 'sub10', etc.
data_dir = "preprocessed_epochs"
# -------------------------------------

# Get all 3 session files for this subject
subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

X_list, y_list = [], []

for file in subject_files:
    try:
        print(f"📂 Loading: {file}")
        X, y = joblib.load(file)
        X_list.append(X)
        y_list.append(y)
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Check if data was loaded
if not X_list:
    print("❌ No data loaded. Check subject ID or file path.")
else:
    # Process subject data
    for i in range(len(X_list)):
        # Reshape & standardize session data
        X_flat = X_list[i].reshape(X_list[i].shape[0], -1)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_flat)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_list[i], test_size=0.2, random_state=42)

        print(f"\n🚀 Running LazyPredict on Session {i+1} of {subject_id}...")
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        print(f"📊 Results for session {i+1} of {subject_id}:\n")
        print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

📂 Loading: preprocessed_epochs/session1_sub6_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session3_sub6_twist_MI_epochs.pkl

🚀 Running LazyPredict on Session 1 of 6...


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 37, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.655792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037353
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.462500 -> initscore=-0.150282
[LightGBM] [Info] Start training from score -0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.658235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036938
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [4]:
# ----------- CONFIGURE THIS ----------
subject_id = "10"  # e.g., 'sub1', 'sub10', etc.
data_dir = "preprocessed_epochs"
# -------------------------------------

# Get all 3 session files for this subject
subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

X_list, y_list = [], []

for file in subject_files:
    try:
        print(f"📂 Loading: {file}")
        X, y = joblib.load(file)
        X_list.append(X)
        y_list.append(y)
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Check if data was loaded
if not X_list:
    print("❌ No data loaded. Check subject ID or file path.")
else:
    # Process subject data
    for i in range(len(X_list)):
        # Reshape & standardize session data
        X_flat = X_list[i].reshape(X_list[i].shape[0], -1)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_flat)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_list[i], test_size=0.2, random_state=42)

        print(f"\n🚀 Running LazyPredict on Session {i+1} of {subject_id}...")
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        print(f"📊 Results for session {i+1} of {subject_id}:\n")
        print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

📂 Loading: preprocessed_epochs/session1_sub10_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session2_sub10_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session3_sub10_twist_MI_epochs.pkl

🚀 Running LazyPredict on Session 1 of 10...


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 41, number of negative: 39
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.671712 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037200
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.512500 -> initscore=0.050010
[LightGBM] [Info] Start training from score 0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.646945 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036789
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.653639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036802
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [ ]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Path to preprocessed .pkl EEG files
                        
subject_id = "1"  # e.g., 'sub1', 'sub10', etc.
data_dir = "preprocessed_epochs"
# -------------------------------------

# Get all 3 session files for this subject
subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

X_all = []
y_all = []

for file in subject_files:
    try:
        print(f"📂 Loading: {file}")
        X, y = joblib.load(file)
        X_all.append(X)
        y_all.append(y)
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Check if data was loaded
if not X_all:
    print("❌ No data loaded. Check subject ID or file path.")
else:
    # Combine all 3 sessions
    X_combined = np.concatenate(X_all, axis=0)
    y_combined = np.concatenate(y_all, axis=0)

    # Flatten and scale
    X_flat = X_combined.reshape(X_combined.shape[0], -1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_flat)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_combined, test_size=0.2, random_state=42)

    print(f"\n🚀 Running LazyPredict on combined sessions for subject {subject_id}...")
    clf = LazyClassifier(verbose=0, ignore_warnings=True)
    models, _ = clf.fit(X_train, X_test, y_train, y_test)

    print(f"📊 Final classification results for subject {subject_id} (3 sessions combined):")
    print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    # Optional: Save to CSV
    #models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']].to_csv(f"{subject_id}_lazy_results.csv")

📂 Loading: preprocessed_epochs/session1_sub1_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session2_sub1_twist_MI_epochs.pkl
📂 Loading: preprocessed_epochs/session3_sub1_twist_MI_epochs.pkl

🚀 Running LazyPredict on combined sessions for subject 1...


  0%|          | 0/32 [00:00<?, ?it/s]

In [1]:
import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyClassifier
import pandas as pd

# -------- CONFIGURE --------
subject_id = "1"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub1_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.682793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037108
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.739040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036750
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.643413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037201
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [2]:
# -------- CONFIGURE --------
subject_id = "6"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub6_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 37, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.661809 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037353
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.462500 -> initscore=-0.150282
[LightGBM] [Info] Start training from score -0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.649633 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036938
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [3]:
# -------- CONFIGURE --------
subject_id = "10"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub10_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 41, number of negative: 39
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.736777 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037200
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.512500 -> initscore=0.050010
[LightGBM] [Info] Start training from score 0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.771098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036789
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.793956 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036802
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [5]:
# -------- CONFIGURE --------
subject_id = "2"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub2_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.778731 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037082
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.750958 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036790
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.882527 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036950
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [2]:
# -------- CONFIGURE --------
subject_id = "3"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub3_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.779506 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037032
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 37, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.656200 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036936
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.462500 -> initscore=-0.150282
[LightGBM] [Info] Start training from score -0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.692731 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037710
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [6]:
# -------- CONFIGURE --------
subject_id = "4"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    #avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    #print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub4_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.674038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037033
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.671089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6034161
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.684160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037011
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [7]:
# -------- CONFIGURE --------
subject_id = "5"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub5_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 41, number of negative: 39
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.653808 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036994
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.512500 -> initscore=0.050010
[LightGBM] [Info] Start training from score 0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 37, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.641678 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037571
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.462500 -> initscore=-0.150282
[LightGBM] [Info] Start training from score -0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.656895 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037085
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [3]:
# -------- CONFIGURE --------
subject_id = "7"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub7_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 42, number of negative: 38
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.682872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036601
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.525000 -> initscore=0.100083
[LightGBM] [Info] Start training from score 0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 38, number of negative: 42
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.647390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036932
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.475000 -> initscore=-0.100083
[LightGBM] [Info] Start training from score -0.100083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [4]:
# -------- CONFIGURE --------
subject_id = "8"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub8_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.748172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036617
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.668343 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037086
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 43, number of negative: 37
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.647159 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036746
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.537500 -> initscore=0.150282
[LightGBM] [Info] Start training from score 0.150282
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [5]:
# -------- CONFIGURE --------
subject_id = "9"  # e.g., "sub01"
data_dir = "preprocessed_epochs"
# ---------------------------

subject_files = sorted(glob(os.path.join(data_dir, f"session*_sub{subject_id}_twist_MI_epochs.pkl")))

results = []

for file in subject_files:
    try:
        print(f"📂 Processing: {file}")
        X, y = joblib.load(file)

        # Flatten and scale
        X = X.reshape(X.shape[0], -1)
        X = StandardScaler().fit_transform(X)

        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LazyPredict
        clf = LazyClassifier(verbose=0, ignore_warnings=True)
        models, _ = clf.fit(X_train, X_test, y_train, y_test)

        models['Session'] = os.path.basename(file)
        results.append(models)

        # Immediately delete large arrays to free memory
        del X, y, X_train, X_test, y_train, y_test, models

    except Exception as e:
        print(f"❌ Failed on {file}: {e}")

# -------------------------
# Aggregate + Save Results
# -------------------------
if results:
    df_all = pd.concat(results)
    avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)

    print(f"\n✅ Averaged LazyPredict Results for {subject_id} (across {len(results)} sessions):")
    print(avg_df[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']])

    avg_df.to_csv(f"{subject_id}_lazy_average_results.csv")
    print(f"📁 Saved: {subject_id}_lazy_average_results.csv")
else:
    print("⚠️ No results generated.")


📂 Processing: preprocessed_epochs/session1_sub9_twist_MI_epochs.pkl


  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.669118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6036961
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 39, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.630622 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037111
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487500 -> initscore=-0.050010
[LightGBM] [Info] Start training from score -0.050010
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.660026 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6037511
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 213071
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [ ]:
#Per-Subject Averaging across all subjects

import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyClassifier
import pandas as pd

# Path to preprocessed .pkl EEG files
data_dir = "preprocessed_epochs"
all_files = sorted(glob(os.path.join(data_dir, "session*_sub*_twist_MI_epochs.pkl")))

# Group .pkl files by subject
subject_sessions = {}
for file in all_files:
    basename = os.path.basename(file)
    subject_id = basename.split("_")[1]  # sub10, sub11 etc.
    subject_sessions.setdefault(subject_id, []).append(file)

# To store all subjects' averaged results
all_subject_averages = []

for subject, files in subject_sessions.items():
    print(f"\n🔄 Processing subject: {subject}")
    session_results = []

    for file in sorted(files):
        try:
            print(f"📂  Loading: {os.path.basename(file)}")
            X, y = joblib.load(file)

            # Flatten and scale
            X = X.reshape(X.shape[0], -1)
            X = StandardScaler().fit_transform(X)

            # Split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # LazyPredict
            clf = LazyClassifier(verbose=0, ignore_warnings=True)
            models, _ = clf.fit(X_train, X_test, y_train, y_test)

            models["Session"] = os.path.basename(file)
            session_results.append(models)

            # Cleanup
            del X, y, X_train, X_test, y_train, y_test, models

        except Exception as e:
            print(f"❌ Failed on {file}: {e}")

    # Compute average over sessions
    if session_results:
        df_all = pd.concat(session_results)
        avg_df = df_all.groupby(df_all.index).mean(numeric_only=True)
        avg_df["Subject"] = subject
        all_subject_averages.append(avg_df)

        print(f"✅ Finished {subject}, averaged across {len(session_results)} sessions")

# -------------------------
# Save All Results
# -------------------------
if all_subject_averages:
    result_df = pd.concat(all_subject_averages)
    result_df.to_csv("lazy_all_subjects_averaged_results.csv")
    print("\n📁 Saved final results to lazy_all_subjects_averaged_results.csv")
else:
    print("\n⚠️ No subject data processed.")


In [ ]:
#Entire Dataset

import os
import numpy as np
import joblib
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyClassifier
import pandas as pd

# Folder with all session .pkl files
data_dir = "preprocessed_epochs"
all_files = sorted(glob(os.path.join(data_dir, "session*_sub*_twist_MI_epochs.pkl")))

# Load and flatten samples one by one to conserve memory
X_list = []
y_list = []

for file in all_files:
    try:
        print(f"📂 Loading: {os.path.basename(file)}")
        X, y = joblib.load(file)
        X = X.reshape(X.shape[0], -1)  # Flatten (channels*time)
        X_list.append(X)
        y_list.append(y)

        # Free memory
        del X, y

    except Exception as e:
        print(f"❌ Error in {file}: {e}")

# Concatenate at once
print("🔧 Concatenating all samples...")
X_all = np.vstack(X_list)
y_all = np.hstack(y_list)

# Free the lists
del X_list, y_list

# Scale
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)

# Split for training/testing
X_train, X_test, y_train, y_test = train_test_split(X_all_scaled, y_all, test_size=0.2, random_state=42)

# LazyPredict classification
print("🧠 Running LazyPredict on full dataset...")
clf = LazyClassifier(verbose=0, ignore_warnings=True)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Save results
#models.to_csv("lazy_full_dataset_results.csv")
#print("\n✅ LazyPredict complete. Results saved to 'lazy_full_dataset_results.csv'.")

# Optional: Show top results
print(models[['Accuracy', 'F1 Score', 'ROC AUC', 'Time Taken']].head())


📂 Loading: session1_sub10_twist_MI_epochs.pkl
📂 Loading: session1_sub11_twist_MI_epochs.pkl
📂 Loading: session1_sub12_twist_MI_epochs.pkl
📂 Loading: session1_sub13_twist_MI_epochs.pkl
📂 Loading: session1_sub14_twist_MI_epochs.pkl
📂 Loading: session1_sub15_twist_MI_epochs.pkl
📂 Loading: session1_sub16_twist_MI_epochs.pkl
📂 Loading: session1_sub17_twist_MI_epochs.pkl
📂 Loading: session1_sub18_twist_MI_epochs.pkl
📂 Loading: session1_sub19_twist_MI_epochs.pkl
📂 Loading: session1_sub1_twist_MI_epochs.pkl
📂 Loading: session1_sub20_twist_MI_epochs.pkl
📂 Loading: session1_sub21_twist_MI_epochs.pkl
📂 Loading: session1_sub22_twist_MI_epochs.pkl
📂 Loading: session1_sub24_twist_MI_epochs.pkl
📂 Loading: session1_sub25_twist_MI_epochs.pkl
📂 Loading: session1_sub2_twist_MI_epochs.pkl
📂 Loading: session1_sub3_twist_MI_epochs.pkl
📂 Loading: session1_sub4_twist_MI_epochs.pkl
📂 Loading: session1_sub5_twist_MI_epochs.pkl
📂 Loading: session1_sub6_twist_MI_epochs.pkl
📂 Loading: session1_sub7_twist_MI_epochs